In [13]:
#OpenCV module
import cv2
#os module for reading training data directories and paths
import os
#numpy to convert python lists to numpy arrays as it is needed by OpenCV face recognizers
import numpy as np#import OpenCV module

#matplotlib for display our images
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
#there is no label 0 in our training data so subject name for index/label 0 is empty
subjects = ["", "Amaan", "Surya","Nehanshu"]

In [15]:
# HAAR
#function to detect face using OpenCV
def detect_face(img):
    #convert the test image to gray image as opencv face detector expects gray images
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #load OpenCV face detector, I am using LBP which is fast
    #there is also a more accurate but slow Haar classifier
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    #let's detect multiscale (some images may be closer to camera than others) images
    #result is a list of faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    
    #if no faces are detected then return original img
    if (len(faces) == 0):
        return None, None
    
#     #under the assumption that there will be only one face,
#     #extract the face area
    (x, y, w, h) = faces[0]
    
    #return only the face part of the image
    return gray[y:y+w, x:x+h],faces[0]


In [16]:
#this function will read all persons' training images, detect face from each image
#and will return two lists of exactly same size, one list 
# of faces and another list of labels for each face
def prepare_training_data(data_folder_path):
    
    #------STEP-1--------
    #get the directories (one directory for each subject) in data folder
    dirs = os.listdir(data_folder_path)
    
    #list to hold all subject faces
    faces = []
    #list to hold labels for all subjects
    labels = []
    
    #let's go through each directory and read images within it
    for dir_name in dirs:
        
        #our subject directories start with letter 's' so
        #ignore any non-relevant directories if any
        if not dir_name.startswith("s"):
            continue;
            
        #------STEP-2--------
        #extract label number of subject from dir_name
        #format of dir name = slabel
        #, so removing letter 's' from dir_name will give us label
        label = int(dir_name.replace("s", ""))
        
        #build path of directory containin images for current subject subject
        #sample subject_dir_path = "training-data/s1"
        subject_dir_path = data_folder_path + "/" + dir_name
        
        #get the images names that are inside the given subject directory
        subject_images_names = os.listdir(subject_dir_path)
        
        #------STEP-3--------
        #go through each image name, read image, 
        #detect face and add face to list of faces
        for image_name in subject_images_names:
            
            #ignore system files like .DS_Store
            if image_name.startswith("."):
                continue;
            
            #build image path
            #sample image path = training-data/s1/1.pgm
            image_path = subject_dir_path + "/" + image_name

            #read image
            image = cv2.imread(image_path)
            
            
            
            #display an image window to show the image 
            cv2.imshow("Training on image...", image)
            cv2.waitKey(100)
            
            #detect face
            face, rect = detect_face(image)
            
            #------STEP-4--------
            #for the purpose of this tutorial
            #we will ignore faces that are not detected
            if face is not None:
                #add face to list of faces
                faces.append(face)
                #add label for this face
                labels.append(label)
            
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    return faces, labels


In [17]:
#let's first prepare our training data
#data will be in two lists of same size
#one list will contain all the faces
#and other list will contain respective labels for each face
print("Preparing data...")
faces, labels = prepare_training_data("training-data")
print("Data prepared")

#print total faces and labels
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))


Preparing data...
Data prepared
Total faces:  705
Total labels:  705


In [18]:
#create our LBPH face recognizer 
recognizer = cv2.face.LBPHFaceRecognizer_create()

#or use EigenFaceRecognizer by replacing above line with 
#recognizer = cv2.face.EigenFaceRecognizer_create()

#or use FisherFaceRecognizer by replacing above line with 
#recognizer = cv2.face.FisherFaceRecognizer_create()


In [19]:
#train our face recognizer of our training faces
recognizer.train(faces, np.array(labels))
#recognizer.save('trainer300.yml')


In [20]:
#function to draw rectangle on image 
#according to given (x, y) coordinates and 
#given width and heigh
def draw_rectangle(img, faces_d):
    if len(faces_d) == 0:
        faces_d = (0,0,0,0)
    (x, y, w, h) = faces_d
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
#function to draw text on give image starting from
#passed (x, y) coordinates. 
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

In [28]:
import cv2 
import numpy as np
# define a video capture object
subjects = ["", "Amaan", "Surya","Nehanshu"]
face_classifier = cv2.CascadeClassifier(r"C:\Users\nehan\Documents\Mega Sync\VJTI\Sum Pro\Haar/cascades/haarcascade_frontalface_default.xml")
#recognizer.read(r"C:\Users\nehan\Documents\Mega Sync\VJTI\Sum Pro\LBPH\trainer300.yml")

vid = cv2.VideoCapture(0)
  
while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Predict part
    im = frame.copy()
    gray_im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(r"C:\Users\nehan\Documents\Mega Sync\VJTI\Sum Pro\LBPH\cascades\haarcascade_frontalface_default.xml")
    #print ("Confidence :",confidence)
    #print("label :",label)
    faces_d = face_cascade.detectMultiScale(gray_im, 1.1, 4)
    print("face Detected: ",faces_d)
    if len(faces_d) == 0:
        faces_d = (0,0,0,0)
    #roi_gray=gray_im[y:y+h,x:x+h]
    for (x,y,w,h) in faces_d:
        roi_gray=gray_im[y:y+h,x:x+h]
        label,confidence=recognizer.predict(roi_gray)
        cv2.rectangle(im,(x,y),(x+w,y+h),(0,255,0),thickness=5)
        cv2.putText(im,subjects[label],(x,y),cv2.FONT_HERSHEY_DUPLEX,3,(255,0,0),6)
    cv2.imshow('frame', im)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  

# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

face Detected:  [[219 190 249 249]]
face Detected:  [[217 187 252 252]]
face Detected:  [[216 185 251 251]]
face Detected:  [[215 184 253 253]]
face Detected:  [[216 181 256 256]]
face Detected:  [[216 183 255 255]]
face Detected:  [[218 183 241 241]]
face Detected:  [[224 155 233 233]]
face Detected:  [[235 151 225 225]]
face Detected:  [[239 151 217 217]]
face Detected:  [[235 150 229 229]]
face Detected:  [[234 151 231 231]]
face Detected:  [[233 160 222 222]]
face Detected:  [[228 155 237 237]]
face Detected:  [[224 162 237 237]]
face Detected:  [[229 166 239 239]]
face Detected:  [[224 160 247 247]]
face Detected:  [[226 165 243 243]]
face Detected:  [[225 167 237 237]]
face Detected:  [[221 161 247 247]]
face Detected:  [[225 164 238 238]]
face Detected:  [[231 167 238 238]]
face Detected:  [[233 162 246 246]]
face Detected:  [[230 163 247 247]]
face Detected:  [[235 175 234 234]]
face Detected:  [[228 167 242 242]]
face Detected:  [[235 172 236 236]]
face Detected:  [[225 167 24